In [ ]:
!pip install gdown
!gdown --id 1If5gkWEfChJHc8v8CCEhGhEeeAqsxKTz

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torch.utils.data import Dataset

from pnpflow.models import UNet

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# define small pretrained unet

net = UNet(
    input_channels=1,
    input_height=28,
    ch=32,
    ch_mult=(1, 2),
    num_res_blocks=2,
    attn_resolutions=(16,),
    resamp_with_conv=True,
).to(device)

net.load_state_dict(
    torch.load('model_final_mnist_dirichlet.pt', 
               map_location=torch.device(device)))
net.eval()

# load testset and define latent distribution
transform = transforms.Compose([transforms.ToTensor()])
diri = torch.distributions.Dirichlet(torch.ones(784))
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

def upsample(x, sf=4):
    '''s-fold upsampler

    Upsampling the spatial size by filling the new entries with zeros

    x: tensor image, NxCxWxH
    '''
    st = 0
    z = torch.zeros(
        (x.shape[0],
         x.shape[1],
         x.shape[2] *
         sf,
         x.shape[3] *
         sf)).type_as(x)
    z[..., st::sf, st::sf].copy_(x)
    return z

def upsample_bicubic(x, sf=2):
    '''s-fold upsampler

    Upsampling the spatial size by filling the new entries with zeros

    x: tensor image, NxCxWxH
    '''
    st = 0
    z = torch.zeros(
        (x.shape[0],
         x.shape[1],
         x.shape[2] *
         sf,
         x.shape[3] *
         sf)).type_as(x)
    z[..., st::sf, st::sf].copy_(x)
    z = F.interpolate(z, scale_factor=sf, mode='bicubic')
    return z


def downsample(x, sf=2):
    '''s-fold downsampler

    Keeping the upper-left pixel for each distinct sfxsf patch and discarding the others

    x: tensor image, NxCxWxH
    '''
    st = 0
    return x[..., st::sf, st::sf]


In [ ]:
## Superresolution Experiment

# load batch and normalize it to the simplex
torch.manual_seed(0)
testloader = torch.utils.data.DataLoader(testset, batch_size=16,
                                          shuffle=True, num_workers=1)

batch,y  = next(iter(testloader))
batch = batch.to(device)
batch = batch/torch.sum(batch, dim = (1,2,3)).unsqueeze(1).unsqueeze(1).unsqueeze(1)


batch_y = batch.clone().to(device)
batch_y = downsample(batch_y, sf = 2) + torch.randn_like(downsample(batch_y, sf = 2))*0.0001

x = diri.sample([16]).to(device)
x = x.view(16,1,28,28)

steps = 300
num_samples = 5

for k in range(steps):
    t = k/steps
    time = torch.tensor(t).to(device)
    x.requires_grad_()
    datafit = torch.sum((downsample(x, sf = 2)-batch_y)**2)*0.5
    grad = torch.autograd.grad(datafit, x)[0]
    x = x.detach()
    z = x - ((1-t))*grad
    x_new = 0
    for _ in range(num_samples):
        z2 = diri.sample([x.shape[0]]).to(device)
        z2 = z2.view(x.shape[0],1,28,28)
        z_new = t*z+(1-t)*z2
        x_new += z_new + (1-t)*net(z_new,time.repeat(x.shape[0]))
    x = x_new/num_samples

print(torch.sum(x, dim = (1,2,3)))
f, axarr = plt.subplots(4,4, figsize = (4,4))
for k in range(4):
    for l in range(4):
        axarr[k,l].imshow(x.cpu().data.numpy()[k*4+l].reshape(28,28), cmap = 'gray')
        axarr[k,l].get_yaxis().set_ticks([])
        axarr[k,l].get_xaxis().set_ticks([])
plt.tight_layout()
#plt.savefig("reco.png")
plt.show()

f, axarr = plt.subplots(4,4, figsize = (4,4))
for k in range(4):
    for l in range(4):
        axarr[k,l].imshow(batch.cpu().data.numpy()[k*4+l].reshape(28,28), cmap = 'gray')
        axarr[k,l].get_yaxis().set_ticks([])
        axarr[k,l].get_xaxis().set_ticks([])
plt.tight_layout()
#plt.savefig("groundtruth.png")
plt.show()


f, axarr = plt.subplots(4,4, figsize = (4,4))
for k in range(4):
    for l in range(4):
        axarr[k,l].imshow(upsample(batch_y, sf = 2).cpu().data.numpy()[k*4+l].reshape(28,28), cmap = 'gray')
        axarr[k,l].get_yaxis().set_ticks([])
        axarr[k,l].get_xaxis().set_ticks([])
plt.tight_layout()
#plt.savefig("measurements.png")
plt.show()

print(torch.sum((batch-x)**2)/len(batch))
print(torch.sum((batch-upsample(batch_y, sf = 2))**2)/len(batch))

In [ ]:
## Denoising Experiment

batch_y = batch.clone().to(device)
torch.manual_seed(1)
batch_y = batch_y+ torch.randn_like(batch_y)*0.001

x = diri.sample([16]).to(device)
x = x.view(16,1,28,28)

steps = 300
num_samples = 5

for k in range(steps):
    t = k/steps
    time = torch.tensor(t).to(device)
    x.requires_grad_()
    datafit = torch.sum((x-batch_y)**2)*0.5
    grad = torch.autograd.grad(datafit, x)[0]
    x = x.detach()
    z = x - ((1-t))*grad
    x_new = 0
    for _ in range(num_samples):
        z2 = diri.sample([x.shape[0]]).to(device)
        z2 = z2.view(x.shape[0],1,28,28)
        z_new = t*z+(1-t)*z2
        x_new += z_new + (1-t)*net(z_new,time.repeat(x.shape[0]))
    x = x_new/num_samples

print(torch.sum(x, dim = (1,2,3)))

f, axarr = plt.subplots(4,4, figsize = (4,4))
for k in range(4):
    for l in range(4):
        axarr[k,l].imshow(x.cpu().data.numpy()[k*4+l].reshape(28,28), cmap = 'gray')
        axarr[k,l].get_yaxis().set_ticks([])
        axarr[k,l].get_xaxis().set_ticks([])
plt.tight_layout()
#plt.savefig("reco_den.png")
plt.show()

f, axarr = plt.subplots(4,4, figsize = (4,4))
for k in range(4):
    for l in range(4):
        axarr[k,l].imshow(batch.cpu().data.numpy()[k*4+l].reshape(28,28), cmap = 'gray')
        axarr[k,l].get_yaxis().set_ticks([])
        axarr[k,l].get_xaxis().set_ticks([])
plt.tight_layout()
#plt.savefig("groundtruth_den.png")
plt.show()


f, axarr = plt.subplots(4,4, figsize = (4,4))
for k in range(4):
    for l in range(4):
        axarr[k,l].imshow(batch_y.cpu().data.numpy()[k*4+l].reshape(28,28), cmap = 'gray')
        axarr[k,l].get_yaxis().set_ticks([])
        axarr[k,l].get_xaxis().set_ticks([])
plt.tight_layout()
#plt.savefig("measurements_den.png")
plt.show()

print(torch.sum((batch-x)**2)/len(batch))
print(torch.sum((batch-batch_y)**2)/len(batch))

In [ ]:
## Inpainting Experiment

batch_y = batch.clone().to(device)
torch.manual_seed(2)
batch_y = batch_y[:,:,:14,:] + torch.randn_like(batch_y[:,:,:14,:])*0.0001

x = diri.sample([16]).to(device)
x = x.view(16,1,28,28)

steps = 300
num_samples = 5

for k in range(steps):
    t = k/steps
    time = torch.tensor(t).to(device)
    x.requires_grad_()
    datafit = torch.sum((x[:,:,:14,:]-batch_y)**2)*0.5
    grad = torch.autograd.grad(datafit, x)[0]
    x = x.detach()
    z = x - ((1-t))*grad
    x_new = 0
    for _ in range(num_samples):
        z2 = diri.sample([x.shape[0]]).to(device)
        z2 = z2.view(x.shape[0],1,28,28)
        z_new = t*z+(1-t)*z2
        x_new += z_new + (1-t)*net(z_new,time.repeat(x.shape[0]))
    x = x_new/num_samples

print(torch.sum(x, dim = (1,2,3)))

f, axarr = plt.subplots(4,4, figsize = (4,4))
for k in range(4):
    for l in range(4):
        axarr[k,l].imshow(x.cpu().data.numpy()[k*4+l].reshape(28,28), cmap = 'gray')
        axarr[k,l].get_yaxis().set_ticks([])
        axarr[k,l].get_xaxis().set_ticks([])
plt.tight_layout()

#plt.savefig("reco_inp.png")
plt.show()

f, axarr = plt.subplots(4,4, figsize = (4,4))
for k in range(4):
    for l in range(4):
        axarr[k,l].imshow(batch.cpu().data.numpy()[k*4+l].reshape(28,28), cmap = 'gray')
        axarr[k,l].get_yaxis().set_ticks([])
        axarr[k,l].get_xaxis().set_ticks([])
plt.tight_layout()
#plt.savefig("groundtruth_inp.png")
plt.show()


f, axarr = plt.subplots(4,4, figsize = (4,4))
for k in range(4):
    for l in range(4):
        axarr[k,l].imshow(torch.cat((batch_y, torch.zeros_like(batch_y)),2).cpu().data.numpy()[k*4+l].reshape(28,28), cmap = 'gray')
        axarr[k,l].get_yaxis().set_ticks([])
        axarr[k,l].get_xaxis().set_ticks([])
plt.tight_layout()
#plt.savefig("measurements_inp.png")
plt.show()

print(torch.sum((batch-x)**2)/len(batch))
print(torch.sum((batch-torch.cat((batch_y,torch.zeros_like(batch_y)),2))**2)/len(batch))